# Pose Prediction From Smartwatch Data
### Part 3

Welcome! This notebook is the second chapter of the hackathon challenge to predict human arm pose from the sensor data of a single smartwatch. In other words, a human wears a smartwatch, and we will try to predict their arm pose from the sensor outputs we can read from the watch.

## Creating a Predictive Model with Machine Learning

In this second chapter, we will have the opportunity to utilize the processed data from the previous chapter and train a Neural Network to generate predictions. The following sections will guide you step-by-step through model architecture design, model training and evaluation, performance analysis, and 3D visualization for better interpretability of the model's performance. 

### 0. Downloading Dependencies (Optional)

We begin by downloading the libraries in this notebook. If you have all of these pre-installed, you may skip this step.

Note: We installed some packages in the first notebook. If you skipped running it, please run step 0 from the first notebook to download dependacies and continute with this notebook. 

In [1]:
!pip install torch
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


### 1. Importing Libraries

Lets import the libraries we will be primarily working with in this notebook. You may notice some similar ones. Let us help you understand why:
- Libraries like Numpy and Pandas are helpful for mathematical functions and data frameworks respectively. Therefore, you would see them almost everywhere.
- Matplotlib is often used to display plots. Using plotly, we will display interactive plots. But why do we need plots for training a machine learning model? You will find out soon. 
- Finally, Torch (PyTorch) provides powerful tools to deploy Neural Networks (NN)

In [2]:
# PyTorch Libraries for Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split    # Scikit-Lean for preparing the train-test data split

import pandas as pd
import numpy as np

# Plotting tools
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# adjust data paths in config.py
import config   
import os    

### 2. Loading Processed Data

Let's load the normalized dataset into the environment that we processed in the previous notebook. You will notice two files in the cache folder. 
1. `normalized_data.csv` contains the normalized dataset
2. `means_stds.csv` contains the means and standard deviations of different columns utilized during normalization.

In [3]:
cache_path = config.paths["cache_path"]
path = os.path.join(cache_path, "normalized_data.csv")
data_normalized = pd.read_csv(path)

### 3. Plotting Functions
Lets get the simpler stuff out of the way first. Since the neural networks will help estimate the X, Y & Z coordinates, we use a similar 3D visualization for representing the outputs by the model and actual labels to gain a deeper understanding of model's performance. 

Secondly, during the training process you will notice the loss decreases with epochs. It is useful to visualize the loss for a number of reasons:
- Overfitting: When model starts remembering the training dataset instead of figuring out a "pattern".
- Performance: To figure out if further training decreases or increases loss 

Note: Epoch refers to the iteration number where the model goes through the entire training data. For instance, 1 epoch indicates that model has passed through the data once. 

For these purposes, we are using helper functions that can be called repeatedly. 

In [4]:
"""
    This section represents the function and helper function required for interactive 3D visualization:
        - The interactive_output_visualization function helps in visualizing the X, Y, Z coordinates in an interactive 3D map. It is also used to display the output and original labels
"""

# Helper function that provides appropriate the approriate length for columns. 
# The duration variable helps in defining the length of continuous data - this helps in visualizing a smaller hand trajectory 
# The randomseed variable helps to visualize the same data without randomizing the selection process
# The remaining functions follow the same variables and analogy
def data_visualization(data, duration, randomseed=False, output=False):
    if output is True:
        dim1, dim2, dim3 = data[:,0], data[:,1], data[:,2]
    else:
        dim1, dim2, dim3 = np.array(data['gt_hand_orig_rua_x']), np.array(data['gt_hand_orig_rua_y']), np.array(data['gt_hand_orig_rua_z'])

    if duration is not None:
        if randomseed:
            np.random.seed(42)
        else: 
            np.random.seed(None)
        reduced_samples = 10 * duration
        start_index = np.random.choice(len(dim1) - reduced_samples + 1)
        dim1 = dim1[start_index : start_index + reduced_samples]
        dim2 = dim2[start_index : start_index + reduced_samples]
        dim3 = dim3[start_index : start_index + reduced_samples]
        
    return dim1, dim2, dim3

def interactive_output_visualization(data, duration=None, randomseed=False, output=False, output_data = None):
    dim1, dim2, dim3 = data_visualization(data, duration, randomseed, output)
    
    trace = go.Scatter3d(x=dim1, y=dim2, z=dim3, mode='markers', marker=dict(size=3, 
                                                                             color='blue', 
                                                                             opacity=0.8),
                                                                             name="Input Points")

    if output:
        dim1, dim2, dim3 = data_visualization(output_data, duration, randomseed, output)
        trace_output = go.Scatter3d(x=dim1, y=dim2, z=dim3, mode='markers', marker=dict(size=3, 
                                                                                    color='red', 
                                                                                    opacity=0.5), 
                                                                                    name='Output Points')
        traces = [trace, trace_output]
        title = 'Input and Output Points in 3D Space'
    else:
        traces = [trace]
        title = 'Values in 3D Space'

    layout = go.Layout(title=title, scene=dict(xaxis=dict(title='X-Dimension'),
                                                                        yaxis=dict(title='Y-Dimension'),
                                                                        zaxis=dict(title='Z-Dimension')))

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

def lossPlot(losses):
    epochs_total = range(1, 1 + len(losses["evaluation"]))

    plt.subplots(1, figsize=(20,5))
    plt.plot(epochs_total, losses["training"], label = "Training Loss")
    plt.plot(epochs_total, losses["evaluation"], label = "Evaluation Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Losses over Epochs')
    plt.legend()
    plt.show()

### 4. Data Loader & Model Architecture

A data loader is an integral part of training a neural network. The data loader assists in selecting the relevant columns assigned as features and labels. We accomplish this task using the PyTorch library. 

In the next portion of the code block, we implement a basic feed-forward architecture comprising an input, two hidden, and an output layer.

In [30]:
"""
    Data Loader: 
        This serves as the framework to load data using pandas and segment out the useful columns (labels and inputs)
"""
gt_columns = [
    "gt_hand_orig_rua_x", "gt_hand_orig_rua_y", "gt_hand_orig_rua_z",
    "gt_larm_6drr_rh_1", "gt_larm_6drr_rh_2", "gt_larm_6drr_rh_3",
    "gt_larm_6drr_rh_4", "gt_larm_6drr_rh_5", "gt_larm_6drr_rh_6",
    "gt_larm_orig_rua_x", "gt_larm_orig_rua_y", "gt_larm_orig_rua_z",
    "gt_uarm_6drr_rh_1", "gt_uarm_6drr_rh_2", "gt_uarm_6drr_rh_3",
    "gt_uarm_6drr_rh_4", "gt_uarm_6drr_rh_5", "gt_uarm_6drr_rh_6"
]

sw_columns = [
    "sw_dt",
    "sw_gyro_x", "sw_gyro_y", "sw_gyro_z",
    "sw_lvel_x", "sw_lvel_y", "sw_lvel_z",
    "sw_lacc_x", "sw_lacc_y", "sw_lacc_z",
    "sw_grav_x", "sw_grav_y", "sw_grav_z",
    "sw_6drr_cal_1", "sw_6drr_cal_2", "sw_6drr_cal_3",
    "sw_6drr_cal_4", "sw_6drr_cal_5", "sw_6drr_cal_6",
    "sw_pres_cal"
]

class MyDataset(Dataset):
    def __init__ (self, data):
        self.data = data
        self.features = self.data.loc[:, sw_columns].values     # All smart watch inputs that contribute towards the outputs
        self.labels = self.data.loc[:, gt_columns[0:3]].values          # The model predicts X, Y, Z coordinates of hand

    def __len__ (self):
        return len(self.data)
    
    def __getitem__ (self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32) 
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return features, label
    
"""
    Neural Network / Model:
        A simple linear architecture based model with 3 layers (including input and output layers). 
"""

class MyModel(nn.Module):
    def __init__ (self, input_size, hidden_size, output_size, batch_size):
        super (MyModel, self).__init__()
        self.conv1_1 = nn.Conv1d(in_channels=input_size, out_channels= int(input_size/2), kernel_size=3, stride=1)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
    
        self.flattened_size = self.calculate_flattened_size(input_size)

        self.fc1 = nn.Linear(self.flattened_size, hidden_size + int(hidden_size/2))
        self.fc2 = nn.Linear(hidden_size + int(hidden_size/2), hidden_size - int(hidden_size/2))
        self.fc3 = nn.Linear(hidden_size - int(hidden_size/2), output_size)
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        
        print("shape: ", x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
    def calculate_flattened_size(self, input_size):
        conv1_output_size = int(input_size / 2)
        maxpool_output_size = conv1_output_size // 2
        flattened_size = maxpool_output_size
        return flattened_size

The next code block provides a basic overview of some of the tunable parameters. We encourage you to play around with these parameters to see what works better for this model architecture and dataset.

In [31]:
batch_size = 1000        
learning_rate = 0.001
num_epochs = 500
test_data_size = 0.2            # Range 0-1. You may increase or decrease the recommended testing data size by updating this variable
hidden_size = 5               # This represents the middle layer of the neural network. You may manually play around with the model architecture by adding more layers

Let's initialize our model with the help of helper functions defined previously. You would also notice the `criterion` and `optimizer` variables that work differently for different tasks. Feel free to play around with them. 

For more information on loss functions and optimizers, visit:  
- https://pytorch.org/docs/stable/nn.html#loss-functions     
- https://pytorch.org/docs/stable/optim.html

In [32]:
# Data loading calls to class and functions
dataset = MyDataset(data_normalized)

train_dataset, test_dataset = train_test_split(dataset, test_size=test_data_size, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialzing the model with inputs
input_size = len(dataset.features[0])
output_size = len(dataset.labels[0])

model = MyModel(input_size, hidden_size, output_size, batch_size)
# Try out different loss functions and optimizers to see what works with different tasks. 
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)     

"""
    If you have GPU resource available, the training process will be faster!
    It's okay if you dont have a GPU. The code works without one as well. 
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MyModel(
  (conv1_1): Conv1d(20, 10, kernel_size=(3,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=5, out_features=7, bias=True)
  (fc2): Linear(in_features=7, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
)

### 5. Model Training & Evaluation

Let's train our data using the defined neural network. The notebook implements the training loop, where batches of data are fed into the model iteratively. During training, the model learns to optimize its parameters to minimize the defined loss function through backpropagation. Training hyperparameters, such as learning rate, batch size, and number of epochs, can be fine-tuned to achieve optimal performance.

Following training, you will notice the evaluation block that evaluates the neural network's performance on a separate test dataset (not included in training) to assess its performance. 

Lastly, the `losses` dictionary is populated to visualize important metrics later in the notebook. 

In [33]:
# Storing data for visually observing the losses
losses = {"training" : [], 
          "evaluation" : [], 
          "labels" : [], 
          "outputs" : [],
          "labels_t" : [], 
          "outputs_t" : []
          }

# Training & Evaluation Porcess
patience = 15                      # Patience based stopping criteria. You may also play around with this number for early or delayed stopping of the model from over-fitting
best_loss = float('inf')
num_epochs_without_improvement = 0

for epoch in range(num_epochs):

    all_outputs_training = []
    all_labels_training = []
    model.train(True)
    running_loss = 0.0                          # To calculates the losses in training for each epoch
    for inputs, labels in train_loader:
        # Transfers data to GPU/CPU
        inputs = inputs.to(device)              
        labels = labels.to(device)
        
        optimizer.zero_grad()                   # Zeros the optimizer before generating output
        print(inputs.shape)
        inputs = inputs.unsqueeze(0) 
        print(inputs.shape)
        inputs = torch.swapdims(inputs, 1, 2)
        print(inputs.shape)
        # exit()
        outputs = model(inputs)                 # Calculates the output
        loss = criterion(outputs, labels)       # Finds loss as per the criteria defined
        loss.backward()                         # Back propogation of loss
        optimizer.step()                        # Updates parameters based on gradients computed duing back propogation
        
        running_loss += loss.item()             # Calculates loss over the training

        all_outputs_training.append(outputs.detach().numpy())  # Append outputs to the list
        all_labels_training.append(labels.detach().numpy())    # Append labels to the list
    
    training_loss = running_loss/len(train_loader)

    """
    Evaluating the model that has been trained (so far). 
    You would notice similar steps as during the training process. The lack of a few lines of code is because we 
    are evaluating the model here and not training it.
    """
    model.train(False)
    model.eval()
    total_loss = 0.0
    all_outputs = []
    all_labels = []

    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            inputs = inputs.unsqueeze(0) 
            inputs = torch.swapdims(inputs, 1, 2)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            all_outputs.append(outputs.cpu().numpy())  
            all_labels.append(labels.cpu().numpy())    
                
    mean_loss = total_loss / len(test_loader)

    if mean_loss < best_loss:
        best_loss = mean_loss
        num_epochs_without_improvement = 0
    else:
        num_epochs_without_improvement += 1
        if num_epochs_without_improvement == patience:
            print("Early stopping triggered. Stopping training.")
            break
    
    # Recording the data values for
    losses["training"].append(training_loss)
    losses["evaluation"].append(mean_loss)
    losses["outputs"] = np.concatenate(all_outputs)
    losses["labels"] = np.concatenate(all_labels)
    
    losses["outputs_t"] = np.concatenate(all_outputs_training)
    losses["labels_t"] = np.concatenate(all_labels_training)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {training_loss}, Smooth L1 Loss: {mean_loss}")

torch.Size([1000, 20])
torch.Size([1, 1000, 20])
torch.Size([1, 20, 1000])
shape:  torch.Size([1, 4990])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4990 and 5x7)

### 6. Performance Analysis & Visualization

The model's training and evaluation losses over an epochs curve help to analyze its learning progress and detect signs of overfitting or underfitting. Analyzing these curves aids in fine-tuning the model's architecture or regularization techniques to achieve a balance between accuracy and generalization.

Since we will be using the neural networks to estimate the X, Y & Z coordinates, a 3D visualization represents the outputs and actual labels to gain a deeper understanding of the model's performance. 

In [ ]:
lossPlot(losses)

In [ ]:
interactive_output_visualization(losses["labels"], duration=4, randomseed=False, output=True, output_data=losses["outputs"])
interactive_output_visualization(losses["labels_t"], duration=4, randomseed=False, output=True, output_data=losses["outputs_t"])

Remove later \\ For manual observation

In [ ]:
print(losses["outputs_t"])
print()
print(losses["labels_t"])

In [ ]:
print(losses["outputs"])
print()
print(losses["labels"])